In [24]:
import os
import numpy as np
import cv2

In [6]:
slides_path = 'slides/'
scenes_path = 'scenes/'

In [1]:
from tensorflow.keras.applications import VGG16
target_shape = (224,224,3)
vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=target_shape,
    pooling='avg')

2021-10-19 13:17:08.326121: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-19 13:17:08.326231: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anton): /proc/driver/nvidia/version does not exist
2021-10-19 13:17:08.327687: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
vgg.trainable = False

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

input_layer = Input(target_shape, dtype = tf.uint8)
y = tf.cast(input_layer, tf.float32)
y = preprocess_input(y)

y = vgg(y)
model = Model(inputs=input_layer, outputs=y)

In [25]:
slides = sorted(os.listdir(slides_path), key=lambda x: int(x.split('.')[0][4:]))
slides = [os.path.join(slides_path, x) for x in slides]
scenes = sorted(os.listdir(scenes_path), key=lambda x: int(x.replace('-','').split('.')[0][6:]))
scenes = [os.path.join(scenes_path, x) for x in scenes]

In [27]:
def load_images(files):
    images = np.zeros((len(files),)+target_shape)
    for i, file in enumerate(files):
        image = cv2.imread(file)
        image = cv2.resize(image, target_shape[:-1])
        images[i] = image
    return images

In [29]:
slides = load_images(slides)
scenes = load_images(scenes)

In [33]:
slides_feature = model.predict(slides)
scenes_feature = model.predict(scenes)

2021-10-19 13:30:33.309552: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
sim = cosine_similarity(slides_feature, scenes_feature)

In [49]:
slides_feature.shape
scenes_feature.shape

(19, 512)

In [57]:
slide_no = 3
print(sim[slide_no])
sim[slide_no].argsort()[::-1]

[0.88272905 0.35927534 0.66365194 0.5111425  0.63732404 0.327524
 0.8211046  0.77427137 0.8107807  0.53656965 0.41712072 0.4170742
 0.37175384 0.76142424 0.79292506 0.80029213 0.7677723  0.72331154
 0.41841915]


array([ 0,  6,  8, 15, 14,  7, 16, 13, 17,  2,  4,  9,  3, 18, 10, 11, 12,
        1,  5])